# Training and hosting SageMaker Models using the Apache MXNet Module API

The **SageMaker Python SDK** makes it easy to train and deploy MXNet models. In this example, we train a simple neural network using the Apache MXNet [Module API](https://mxnet.incubator.apache.org/api/python/module.html) and the MNIST dataset. The MNIST dataset is widely used for handwritten digit classification, and consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). The task at hand is to train a model using the 60,000 training images and subsequently test its classification accuracy on the 10,000 test images.

### Setup

First we need to define a few variables that will be needed later in the example.

In [ ]:
from sagemaker import get_execution_role
import json
import boto3
import time
import os
import time
import tarfile
cf = boto3.client('cloudformation')
s3 = boto3.client('s3')
sns = boto3.client('sns')
step = boto3.client('stepfunctions')
sagemaker = boto3.client('sagemaker-runtime')




with open('../config.json') as json_file:  
    config = json.load(json_file)
StackName=config["StackName"]

result=cf.describe_stacks(
    StackName=StackName
)
outputs={}
for output in result['Stacks'][0]['Outputs']:
    outputs[output['OutputKey']]=output['OutputValue']

with tarfile.open("script.tar.gz", "w:gz") as tar:
    tar.add(os.getcwd(),arcname="")

s3.upload_file("script.tar.gz",outputs["CodeBucket"],"script.tar.gz")
#IAM execution role that gives SageMaker access to resources in your AWS account.
#We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

### Update SageBuild Template

In [ ]:
result=cf.describe_stacks(
    StackName=StackName
)

params={}
for param in result['Stacks'][0]['Parameters']:
    params[param['ParameterKey']]=param['ParameterValue']
    
params["ConfigPresetType"]="MXNET"
params["HyperParameters"]=json.dumps({'learning_rate': 0.1})
params["TrainEntryPoint"]="mnist.py"
params["TrainSourceFile"]="s3://{}/script.tar.gz".format(outputs["CodeBucket"])
params["HostEntryPoint"]="mnist.py"
params["HostSourceFile"]="s3://{}/script.tar.gz".format(outputs["CodeBucket"])

Parameters=[ {"ParameterKey":param, "ParameterValue":params[param]} for param in params]
cf.update_stack(
    StackName=StackName,
    UsePreviousTemplate=True,
    Parameters=Parameters,
    Capabilities=['CAPABILITY_NAMED_IAM']
)
waiter = cf.get_waiter('stack_update_complete')
waiter.wait(
    StackName=StackName,
    WaiterConfig={
        'Delay': 10,
        'MaxAttempts': 123
    }
)


### The training script

The ``mnist.py`` script provides all the code we need for training and hosting a SageMaker model. The script we will use is adaptated from Apache MXNet [MNIST tutorial (https://mxnet.incubator.apache.org/tutorials/python/mnist.html).

In [ ]:
!cat mnist.py

### Start Train/Deploy pipeline

In [ ]:
%%time
result=sns.publish(
    TopicArn=outputs['LaunchTopic'],
    Message="{}" #message is not important, just publishing to topic starts build
)
print(result)
sleep(5)
#list all executions for our StateMachine to get our current running one
result=step.list_executions(
    stateMachineArn=outputs['StateMachine'],
    statusFilter="RUNNING"
)['executions']

if len(result) > 0:
    response = step.describe_execution(
        executionArn=result[0]['executionArn']
    )
    status=response['status']
    print(status,response['name'])
    #poll status till execution finishes
    while status == "RUNNING":
        print('.',end="")
        sleep(5)
        status=step.describe_execution(executionArn=result[0]['executionArn'])['status']
    print()
    print(status)
else:
    print("no running tasks")

### Making an inference request

The request handling behavior of the Endpoint is determined by the ``mnist.py`` script. In this case, the script doesn't include any request handling functions, so the Endpoint will use the default handlers provided by SageMaker. These default handlers allow us to perform inference on input data encoded as a multi-dimensional JSON array.

To see inference in action, draw a digit in the image box below. The pixel data from your drawing will be loaded into a ``data`` variable in this notebook. 

*Note: after drawing the image, you'll need to move to the next notebook cell.*

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

Now we can classify the handwritten digit:

In [ ]:
result=sagemaker.invoke_endpoint(
    EndpointName=outputs['SageMakerEndpoint'],
    Body=json.dumps(data),    
    ContentType="application/json",
    Accept="application/json"
)

response =json.loads(result['Body'].read().decode('utf-8'))
labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))